<h1>Demo description</h1>

**Presenter**: Ben Priest

In this demonstration, we examine how to implement gradient descent in Python. We will use the Ackley function as an example test for optimization example, making clear where another function may be swapped out.

<h3>Note</h3>

The purpose of providing this code is to make a python version of some of the Matlab code that we've worked on in class available to everyone. Please bear in mind the honor principle when doing homework. 

I also provide no guarantees that this code is bug-free.

In [ ]:
%matplotlib inline

import cProfile as profile
import itertools
import random
import matplotlib.pyplot as plt
import numpy as np
import operator

from mpl_toolkits.mplot3d import Axes3D

<h2>Ackley function</h2>

These functions all compute the ackley function, a common multidimensional test surface for optimization algorithms. This implementation is agnostic with respect to the number of dimensions over which it is computed, although typically 2 dimensions are used for ease of visualization. 

add text 

We presuppose the values of the parameters for the ackley function in the function heading as keyworded arguments. Different values may be provided at execution by passing new values to the specified keywords.

<h3>vanilla_ackley</h3>

This implementation is the same as the standard Matlab implementation examined in ENGS 104 on 9/24/15. 

In [ ]:
def vanilla_ackley(x, a=20.0, b=0.2, c=2.0*np.pi):
    s1 = 0
    s2 = 0
    n = len(x)
    for i in range(n):
        s1 = s1 + x[i]**2.0
        s2 = s2 + np.cos(c*x[i])
    return -a*np.exp(-b*np.sqrt(s1/n)) - np.exp(s2/n) + a + np.exp(1)

<h3>pythonic_ackley</h3>

We can improve on the performance of vanilla ackley be computing it pythonically. Performance suffers in vanilla ackley for large values of n (dimensions of x) because of the overhead introduced in the for loop. This is especially true in Matlab, because of how Matlab implements control code. 

High-dimensional x are unlikely for this particular application, but it is still a useful example for demonstrating how to improve the efficiency of certain computations using functional constructs. 

In [ ]:
def pythonic_ackley(x, a=20.0, b=0.2, c=2.0*np.pi):
    n = len(x)
    s1 = np.sum(np.array([i**2.0 for i in x]))
    s2 = np.sum(np.array([np.cos(c*i) for i in x]))
    return -a*np.exp(-b*np.sqrt(s1/n)) - np.exp(s2/n) + a + np.exp(1)

<h3>Efficiency Check</h3>

Let's check the efficiency of the two functions with respect to the dimension of the input. We will profile each implementation's performance evaluating 100 random samples, each of dimension 5000. 

While it is not a perfect comparison, I clocked the Matlab code from class performing the same calculations in about 3.084s.  

In [ ]:
dim = 5000
n_samples = 100

Y = [[random.uniform(-5,5) for i in range(dim)] for j in range(n_samples)]

In [ ]:
profile.run('map(vanilla_ackley,Y)')

In [ ]:
profile.run('map(pythonic_ackley,Y)')

<h3>Sanity Check</h3>

We'll now make sure that these two compute the same function.

In [ ]:
# apply vanilla_ackley to each of the samples
vs = map(vanilla_ackley, Y)

# apply pythonic_ackley to each of the samples
ps = map(pythonic_ackley, Y)

# check for equivalence
all(map(lambda x: x[0] == x[1], zip(vs,ps)))

<h3>Plotting the results</h3>

Similar to in class, we'll verify out function by plotting the resulting surface over $[-5,5] \times [-5, 5]$. Using the standard matplotlib scipy plotting library, this is much the same as plotting in Matlab. 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = y = np.arange(-5.0, 5.0, 0.05)
X, Y = np.meshgrid(x, y)
zs = np.array([pythonic_ackley([x,y]) for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Ackley Value')

plt.show()

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
x = y = np.arange(-5.0, 5.0, 0.05)
X, Y = np.meshgrid(x, y)
zs = np.array([pythonic_ackley([x,y]) for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)

fig = plt.figure()
CS = plt.contour(X, Y, Z)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Simplest default with labels')

#ax.plot_surface(X, Y, Z)

#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Ackley Value')

plt.show()

<h2>Gradient Function</h2>

We can define a gradient estimation function that can operate agnostically of the actual underlying function. So, we can use the same gradient estimation code in conjunction with either of the above implementations of the ackley function, or with any other function $f$ such that $f: \mathbb{R}^n \rightarrow \mathbb{R}$.

In [ ]:
def gradient(f, x, eps=0.0001):
    return [(f([x[j] + eps if j == i else x[j] for j in range(len(x))]) - f(x))/eps for i in range(len(x))]

<h3>Why does this work?</h3>

We pass a function **f** to **gradient**, as well as a vector-valued test point **x** and some scalar epsilon **eps**. Gradient computes a first-order approximation of the partial derivative with respect to each element of **x** and returns a vector of these values.

In [ ]:
gradient(pythonic_ackley, [0.1,-0.6])

<h2>Gradient Descent</h2>

We can now implement gradient descent in the usual way. We will here implement vanilla gradient descent, not varying the $\epsilon$ or $\alpha$ update parameters, and also not checking for convergence. This approach can be easily extended to support a particle swarm or simulated annealing approach.

In [ ]:
def vanilla_gd(f, x, epsilon=0.0001, alpha=0.001, its=1000):
    for i in range(its):
        x = map(operator.sub, x, alpha * np.array(gradient(f, x, eps=epsilon)))
    return x

In [ ]:
vanilla_gd(pythonic_ackley, [0.3, -0.2], alpha=0.001, its=50000)

In [ ]:
pythonic_ackley([0,0])

In [ ]:
pythonic_ackley([0.013573552847734038, -0.013636680083916997])

<h3>Other functions</h3>

We can easily swap in other functions and analyze them. The only requirements are stated above.

In [ ]:
def sphere(x):
    return np.sum(np.array([i**2 for i in x]))

In [ ]:
vanilla_gd(sphere, [322, -33], alpha=0.01, its=50000)